# 토큰 플레이그라운드 (Jupyter)

- 이 노트북은 토큰이 뭔지 “눈으로 확인”하며 이해하기 위한 간단한 실습용입니다.
- 아래 변수 `sample_text`, `toy_vocab`를 직접 바꿔가며 각 토큰화 방식 결과를 비교하세요.
- `vocab`는 `vocabulary(어휘집)`의 줄임말로, “모델이 아는 토큰들의 목록”을 뜻합니다.
- 설치가 필요 없다면 기본 파이썬으로도 동작하지만, 노트북이 더 편합니다.

In [4]:
# 실험할 문장과 장난감 서브워드 어휘(더 짧은 예시)
sample_text = "해리포터는 마법사야!"

# 아주 간단한 장난감 vocab: 자주 등장한다고 가정한 조각들
toy_vocab = [
    "해리포터", "는", "마법사", "야", "!", " ",
]

print("문장:", sample_text)
print("toy_vocab 수:", len(toy_vocab))


문장: 해리포터는 마법사야!
toy_vocab 수: 6


### 용어 미니사전
- **토큰(Token)**: 문장을 모델이 다루기 쉬운 작은 단위(단어/부분 단어 등)
- **어휘집(Vocabulary, vocab)**: 모델이 아는 토큰의 목록
- **토크나이저(Tokenizer)**: 문장을 토큰 목록으로 바꾸는 도구

In [5]:
import re
from typing import List


def tokenize_whitespace(text: str) -> List[str]:
    return text.split()


def tokenize_characters(text: str) -> List[str]:
    return list(text)


def tokenize_punct_aware(text: str) -> List[str]:
    pattern = r"[A-Za-z0-9_]+|[\uAC00-\uD7A3]+|[^\sA-Za-z0-9_\uAC00-\uD7A3]"
    return re.findall(pattern, text)


def greedy_subword_tokenize(text: str, vocab: List[str]) -> List[str]:
    if not text:
        return []
    sorted_vocab = sorted(set(vocab), key=len, reverse=True)
    tokens: List[str] = []
    i = 0
    while i < len(text):
        matched = False
        for token in sorted_vocab:
            if text.startswith(token, i):
                tokens.append(token)
                i += len(token)
                matched = True
                break
        if not matched:
            tokens.append(text[i])
            i += 1
    return tokens


## Step 1. 공백 기준 토큰화
- 바로 아래 셀을 실행해 결과를 확인하세요.
- 헷갈리면 이 셀(설명)만 다시 읽고, 아래 코드만 실행하면 됩니다.

In [7]:
print("문장:", sample_text)
ws = tokenize_whitespace(sample_text)
print("[공백 기준] 토큰 수:", len(ws))
print(ws)


문장: 해리포터는 마법사야!
[공백 기준] 토큰 수: 2
['해리포터는', '마법사야!']


## Step 2. 문자 기준 토큰화
- 한 글자씩 쪼갰을 때 길이가 어떻게 달라지는지 확인하세요.
- 긴 리스트가 부담되면 미리보기(`[:50]`)만 봐도 됩니다.


In [8]:
chars = tokenize_characters(sample_text)
print("[문자 기준] 토큰 수:", len(chars))
print(chars[:50], ("..." if len(chars) > 50 else ""))


[문자 기준] 토큰 수: 11
['해', '리', '포', '터', '는', ' ', '마', '법', '사', '야', '!'] 


## Step 3. 구두점 인지 토큰화(간단)
- 단어/한글 블록과 기호를 분리합니다.
- 문장 끝의 마침표, 느낌표, 영어/숫자 등이 어떻게 나뉘는지 보세요.


In [9]:
pn = tokenize_punct_aware(sample_text)
print("[구두점 인지] 토큰 수:", len(pn))
print(pn)


[구두점 인지] 토큰 수: 3
['해리포터는', '마법사야', '!']


## Step 4. 장난감 서브워드
- `toy_vocab`에 있는 조각들을 우선적으로 길게 매칭합니다.
- `toy_vocab`에 조각을 추가/삭제하며 토큰 수 변화를 관찰하세요.


In [10]:
sw = greedy_subword_tokenize(sample_text, toy_vocab)
print("[서브워드] 토큰 수:", len(sw))
print(sw)

print("\n팁) toy_vocab에 자주 나오는 덩어리를 넣으면 토큰 수가 줄어듭니다.")


[서브워드] 토큰 수: 6
['해리포터', '는', ' ', '마법사', '야', '!']

팁) toy_vocab에 자주 나오는 덩어리를 넣으면 토큰 수가 줄어듭니다.


### 사용 팁
- 위 첫 번째 셀의 `sample_text`, 두 번째 줄의 `toy_vocab`를 바꾸고 이 셀 위쪽부터 순서대로 실행하세요.
- `toy_vocab`에 자주 등장하는 조각을 추가하면, 서브워드 결과 토큰 수가 줄어드는 경향을 볼 수 있습니다.
- 이모지나 특수문자를 `sample_text`에 넣고 `구두점 인지` 결과가 어떻게 변하는지도 확인해보세요.